출처 : https://bab2min.tistory.com/570  
출처 : https://wiserloner.tistory.com/932

In [ ]:
!pip install networkx
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 46.1MB/s 
     |████████████████████████████████| 92kB 13.0MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Komoran
import networkx
import re
import os
import math

### **RawTextReader**
file path를 읽어, Iterate할 때 문장 단위로 반환해주는 클래스


In [ ]:
class RawTextReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile("/n")

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in ch:
                yield s

### **TextRank**

문서 내 문장들의 TR을 구하는 클래스

**```loadSents```** 

문장을 읽어 품사 태깅 값 저장한 후, 문장 간 유사도를 비교하여 일정 이상 유사할 경우 ```dictBiCount[i,j]```에 저장하는 함수이다.    

- ```dictCount``` : 문장 저장  
- ```sentSet``` : 문장의 품사 태깅한 값 저장 (명사, 동사, 형용사만 추출)  
- ```dictBiCount``` : 현재 문장 i와 이전 문장 j의 유사도를 비교하여, threshold를 넘을 경우 ```dictBiCount[i,j]```에 유사도 값 저장    
  

유사도는 각 문장 sentSet 간의 교집합 원소 개수를 n이라 했을 때, 아래 식을 계산한 값이다.  
아래 유사도는 자카드 유사도 계산식에 따라 계산되었으며, 0에서 1 사이 값을 가진다.  
```
similarity = n / float(len(A) + len(B) - n) / (math.log(len(A)+1) * math.log(len(B)+1))
```
  
**```build```**
 
문장의 개수만큼 노드를 생성하고, i 노드와 j 노드 사이 간선에 아래 식을 계산한 값을 부여한다.  

```
weight = dictBiCount[i,j]*self.coef + (1-self.coef) # self.coef = 1.0 (미지정)
```

   
**```rank```**  
생성한 그래프를 바탕으로 각 노드의 ```pagerank```를 구한다.  


**```summarize```**  
입력 값 ratio 값에 따라 ```int(문장 개수 * ratio)``` 만큼 중요 문장을 이어붙여 반환한다. 


### **품사 태깅 (POS Tagging, part-of-speech tagging)**

각 단어의 품사를 태깅하는 프로세스.

Komoran는 형태소 분석을 지원하며, 이를 사용하여 품사 태깅 진행  
```NNG``` : 일반 명사   
```NNP``` : 고유 명사  
```VV``` : 동사  
```VA``` : 형용사  

articles/Pretty-Data 경로 내 있는 데이터는, 문장단위 구분만을 수행한 원본 데이터이며 아래 ```sent```와 같다.

In [ ]:

sent = "청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다"
tagger = Komoran()
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])

tokenizer = lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent))
fil = tokenizer(sent)

' '.join([ token for (token, pos) in list(filter(None, fil))])

'청주 출신 청주 고와 경찰대 졸업 송해 영 계장 경위 경찰 입문 충북 청 기획 예산 계장 경무 계장 홍보 계장 역임'

articles/Preprocessed-Data 경로 내 있는 데이터는, 문장단위 구분, 중복 공백 및 특수문자 삭제, 불용어 삭제를 수행한 전처리 된 데이터이며 아래 출력과 같다.


TF-IDF를 이용하여 문장 간 유사도를 구하기 위해 아래 전처리 된 데이터를 사용한다.  
 ```Document``` 클래스를 이용해 원본과 전처리 데이터를 매핑하여 출력한다.

In [ ]:
fil = tokenizer(sent)
s= ' '.join([noun for (noun, pos) in list(fil)])
s

'청주 출신 청주 고와 경찰대 졸업 송해 영 계장 경위 경찰 입문 충북 청 기획 예산 계장 경무 계장 홍보 계장 역임'

In [ ]:
class Document:
    def __init__(self, originSentenceIter, procSentenceIter):
        self.originSents = list(filter(None, originSentenceIter))
        self.procSents = list(filter(None, procSentenceIter))
    
    def getOriginSet(self):
        return self.originSents

    def getSentsZip(self):
        return zip(self.originSents, self.procSents)

In [ ]:
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}

        self.tfidf_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = {}

    
    def loadSents(self, document, tokenizer, similarity = 'jaccard'):
        import math
        def jaccard_similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))

        sentSet = []
        for origin, proc in document.getSentsZip():
            tagged = set(filter(None, tokenizer(proc)))
            if len(tagged) < 2: continue
            self.dictCount[len(self.dictCount)] = origin
            sentSet.append(tagged)
        
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(document.getOriginSet())


        if similarity is not 'jaccard' :
            sents_distances = (self.tfidf_matrix * self.tfidf_matrix.T)
            sents_distances_matrix = sents_distances.toarray()

            for i in range(len(self.dictCount)):
                for j in range(i+1, len(self.dictCount)):
                    s = sents_distances_matrix[i, j]

                    if s < self.threshold: continue
                    self.dictBiCount[i, j] = s
        
        else: 
            for i in range(len(self.dictCount)):
                for j in range(i+1, len(self.dictCount)):
                    s = jaccard_similarity(sentSet[i], sentSet[j])

                    if s < self.threshold: continue
                    self.dictBiCount[i, j] = s
    
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

기사 파일 경로 지정



> ```articles/Origin-Data``` :  크롤링한 원본 데이터, (기사 제목, 본문, 언론사를 포함한다.)  
> ```articles/Preprocessed-Data``` :  전처리를 마친 데이터 (중복 공백, 특수문자, 불용어 제거)  
> ```articles/Pretty-Data``` :  원본 데이터에서 본문을 문장 단위로 구분한 데이터  
> ```artitlces/StopWordList```:  불용어 리스트



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
BASE_DIR = "/content/gdrive/My Drive/Colab Notebooks/ETRI_Article_Summarizer/Text-preprocessing-Data/articles"
PREPROCESSED_PATH = os.path.join(BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(BASE_DIR,"Pretty-Data")
ORIGIN_PATH = os.path.join(BASE_DIR,"Origin-Data")
SWORDS_PATH = os.path.join(BASE_DIR, "StopWordList.txt")

In [ ]:
media_list = os.listdir(PREPROCESSED_PATH)

origin_article_list = os.listdir(os.path.join(PRETTY_PATH, media_list[0]))
proc_article_list = os.listdir(os.path.join(PREPROCESSED_PATH, media_list[0]))

origin_article_path = os.path.join(os.path.join(PRETTY_PATH, media_list[0]), origin_article_list[0])
proc_article_path = os.path.join(os.path.join(PREPROCESSED_PATH, media_list[0]), proc_article_list[0])

불용어로 StopWordList.txt 파일 내 있는 용어들을 사용하였으며 이는 품사 태깅에서 제외될 항목이다.  
rank 함수를 통해 얻은 각 문서의 TR값에 따라 정렬할 경우, 중요한 순서대로 나열된다.  



In [ ]:
def getStopWord(filename):
    swords = []
    tagger = Komoran()
    with open(filename, 'r', encoding='utf-8') as f:
        swords = f.readlines()
        swords = [tag for sword in swords for tag in tagger.pos(sword.strip()) if tag[1] in ('NNG', 'NNP', 'VV', 'VA')]

    return swords

In [ ]:
# stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
stopword = getStopWord(SWORDS_PATH)

자카드 유사도 계산식을 사용하여 문장 간 유사도를 구한 경우

*출력된 문장은 원본 문장이다.*

In [ ]:
tr = TextRank()
tagger = Komoran()

tr.loadSents(Document(RawTextReader(origin_article_path), RawTextReader(proc_article_path)),
             lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))

tr.build()

ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

4	0.31316907533790084	 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.
2	0.27082082905736027	 괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다.
1	0.179576847738044	 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다.
0	0.1489460919231845	(청주=뉴스1) 조준영 기자 = 충북 경찰에서 4년 연속 2명의 총경 승진자가 나왔다.
3	0.08748715594351042	 이후 청주 흥덕경찰서 생활안전과장과 경비교통과장, 충청 보안수사대장, 생활질서계장 등을 지냈다.
 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.


TFIDF matrix를 이용하여 문장과 문장 간 유사도를 확인할 수 있다.  
유사도를 기반으로 edge에 weight를 부여하여 pagerank 알고리즘을 적용한다.

In [ ]:
tr = TextRank()
tagger = Komoran()

tr.loadSents(Document(RawTextReader(origin_article_path), RawTextReader(proc_article_path)), 
             lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)), similarity='tfidf')

tr.build()
ranks = tr.rank()

for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

4	0.3494246897041189	 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.
2	0.26416847984523795	 괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다.
3	0.14648513601180366	 이후 청주 흥덕경찰서 생활안전과장과 경비교통과장, 충청 보안수사대장, 생활질서계장 등을 지냈다.
1	0.14170317753421804	 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다.
0	0.09821851690462159	(청주=뉴스1) 조준영 기자 = 충북 경찰에서 4년 연속 2명의 총경 승진자가 나왔다.
 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.


#### **Gensim 패키지를 이용한 문장 간 유사도 측정**  

gensim 라이브러리를 사용하는 경우 아래와 같다.  
기사 본문 내용에 따라 단어 사전을 구성하고, TfidfModel을 생성한다.  
이때 similarities를 통해 문장 간 유사도 값을 가지는 행렬을 구할 수 있다.   

In [ ]:
import gensim
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
class Gensim_TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}

        self.tfidf_model = None
        self.dictionary = corpora.Dictionary()
        self.BoW_corpus = []
    
    
    def loadSents(self, sentIter, tokenizer= None):

        sentTokenList = []
        for sent in sentIter :

            if tokenizer : sent = ' '.join([ token for (token, pos) in list(filter(None, tokenizer(sent)))])
            sentToken = simple_preprocess(sent)

            if len(sentToken) < 2: continue

            sentTokenList.append(sentToken)
            self.dictCount[len(self.dictCount)] = sent
        
        self.BoW_corpus = [self.dictionary.doc2bow(token, allow_update=True) for token in sentTokenList]
        self.tfidf_model = models.TfidfModel(self.BoW_corpus, smartirs='ntc')
        
        tfidf = self.tfidf_model[self.BoW_corpus]
        index = similarities.MatrixSimilarity(tfidf)
        sims = index[tfidf]

        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = sims[i,j]

                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
    
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

In [ ]:
gensim_tr = Gensim_TextRank()

gensim_tr.loadSents(RawTextReader(origin_article_path))

gensim_tr.build()
gensim_ranks = gensim_tr.rank()

for k in sorted(gensim_ranks, key=gensim_ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(gensim_ranks[k]), str(gensim_tr.dictCount[k])]))
print(gensim_tr.summarize(0.2))

4	0.3735603893790466	 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.
2	0.2687020691245161	 괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다.
1	0.1753532582769251	 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다.
3	0.10773734635052537	 이후 청주 흥덕경찰서 생활안전과장과 경비교통과장, 충청 보안수사대장, 생활질서계장 등을 지냈다.
0	0.04552072327675353	(청주=뉴스1) 조준영 기자 = 충북 경찰에서 4년 연속 2명의 총경 승진자가 나왔다.
5	0.02912621359223301	reason@news1.
 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.
